In [1]:
from utils.build_features import build_features
from utils.utils import get_klines, add_ys, merge_klines, client_intervals, merge_assets
from binance.client import Client
import pandas as pd
import numpy as np
import copy

from models.random_forest import RandomForest

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Hyper parameters:
CUTOFF = 0.15  # in percents. The minimal value of ascending
N_ESTIMATORS = 100


In [3]:
s_date = '01 Jan, 2019'
e_date = '02 Jan, 2019'
symbols = ['BTCUSDT', 'ETHUSDT']
pull_interval = '5m'
data_interval = '30m'
merging = 6  # Should be equal to data_interval/pull_interval


In [49]:
kl_file_names = []
for symbol in symbols:
    kl_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
    kl_file_names.append(kl_f_name)
    # small_kl = get_klines(s_date, e_date, symbol, client_intervals[pull_interval])
    # klines = merge_klines(
    #     small_kl,
    #     merge_amount=merging)
    # klines.to_csv('klines/30M/' + kl_f_name, index=False)

In [6]:
# kl_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
# small_kl = get_klines(s_date, e_date, symbol, client_intervals[pull_interval])
# klines = merge_klines(
#     small_kl,
#     merge_amount=merging)
# klines.to_csv('klines/30M/' + kl_f_name, index=False)
# klines = pd.read_csv('klines/30M/' + kl_f_name)


In [7]:
print(klines.shape)
klines.head()

(283, 8)


,timestamp,open,high,low,close,volume,#trades,R^2
0,1.546301e+12,3701.23,3703.72,3689.88,3694.20,322.716816,2398.0,0.146206
1,1.546301e+12,3696.30,3705.00,3689.88,3701.41,295.320636,2414.0,0.267947
2,1.546301e+12,3692.34,3705.00,3690.56,3700.55,263.324379,2432.0,0.348681
3,1.546302e+12,3697.91,3705.00,3690.56,3702.39,303.620435,2662.0,0.833680
4,1.546302e+12,3693.44,3710.55,3690.56,3708.60,351.464284,2840.0,0.905247


In [50]:
features_file_names = []
for kl_f in kl_file_names:
    features_f_name = kl_f
    klines = pd.read_csv('klines/30M/' + kl_f)
    features_file_names.append(features_f_name)
    features = build_features(klines)
    features.to_csv('features/30M/' + features_f_name, index=False)


In [52]:
features = merge_assets(symbols, features_file_names, '30M')

['BTCUSDT', 'ETHUSDT']
['BTCUSDT_01 Jan, 2019_TO_02 Jan, 2019.csv', 'ETHUSDT_01 Jan, 2019_TO_02 Jan, 2019.csv']
(267, 247)


In [38]:
features.columns

Index(['timestamp', 'BTCUSDT_open', 'BTCUSDT_high', 'BTCUSDT_low',
       'BTCUSDT_close', 'BTCUSDT_volume', 'BTCUSDT_#trades', 'BTCUSDT_R^2',
       'BTCUSDT_last_close', 'BTCUSDT_close_ratio',
       ...
       'ETHUSDT_volume_max_16', 'ETHUSDT_close_ratio_mean_16',
       'ETHUSDT_close_ratio_min_16', 'ETHUSDT_close_ratio_max_16',
       'ETHUSDT_open_ratio_16', 'ETHUSDT_close_ratio_16',
       'ETHUSDT_high_ratio_16', 'ETHUSDT_low_ratio_16',
       'ETHUSDT_volume_ratio_16', 'ETHUSDT_#trades_ratio_16'],
      dtype='object', length=247)

In [54]:
# features_y = copy.deepcopy(features)
# features_y['y'] = features_y['close_ratio'].shift(-1)
# features_y = features_y.iloc[:-1]
symbol_to_pred = 'BTCUSDT'
features_y = add_ys(features, CUTOFF, symbol_to_pred)

In [55]:
np.max(features_y['y_bins'])

5

In [56]:
x = copy.deepcopy(features_y)
x = x[x['y_bins'] >= 0]
x[['BTCUSDT_close','y', 'y%', 'y_R^2', 'y*r2', 'y_bins']].head(100)


,BTCUSDT_close,y,y%,y_R^2,y*r2,y_bins
27,3695.96,1.002716,0.27164,0.822126,1.002233,1
66,3709.25,1.002809,0.28092,0.662378,1.001861,1
132,3712.14,1.002901,0.29013,0.568468,1.001649,1
175,3687.12,1.001752,0.17520,0.891312,1.001562,1
190,3678.04,1.002768,0.27678,0.584565,1.001618,1
216,3683.49,1.003364,0.33636,0.794859,1.002674,1
218,3696.73,1.001899,0.18990,0.876211,1.001664,1
220,3700.13,1.003173,0.31729,0.840675,1.002667,1
221,3711.87,1.002012,0.20125,0.843315,1.001697,1
230,3719.00,1.002958,0.29578,0.689874,1.002041,1


In [57]:
l = features.shape[0]
X_train = np.array(features.iloc[:int(0.8 * l)].drop('timestamp', axis=1))
X_valid = np.array(features.iloc[int(0.8 * l):].drop('timestamp', axis=1))
y_train = np.array(features_y['y_bins'].iloc[:int(0.8 * l)])
y_valid = np.array(features_y['y_bins'].iloc[int(0.8 * l):])
df_valid = features.iloc[int(0.8 * l):]
df_valid_y = features_y.iloc[int(0.8 * l):]


In [67]:
l = features.shape[0]
X_train = np.array(features.drop('timestamp', axis=1))
X_valid = np.array(features.drop('timestamp', axis=1))
y_train = np.array(features_y['y_bins'])
y_valid = np.array(features_y['y_bins'])
df_valid = features
df_valid_y = features_y

In [69]:
model = RandomForest(n_estimators=N_ESTIMATORS)

In [70]:
model.fit(
    X=X_train,
    y=y_train)

In [71]:
df_valid_y['predictions'] = model.predict(df_valid.drop('timestamp', axis=1))
pred_file_name = 'try.csv'
df_valid_y.to_csv('predictions/'+pred_file_name, index=False)

[-1  0  1  5]
[-0.16666667  0.          0.16666667  0.83333333]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[0.98 0.   0.02 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]


[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.   0.01]
[1. 0. 0. 0.]
[0.94 0.   0.05 0.01]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.   0.01]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]


[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[0.33 0.   0.67 0.  ]
[0.89 0.   0.11 0.  ]
[0.94 0.   0.06 0.  ]
[0.97 0.   0.03 0.  ]
[0.98 0.   0.02 0.  ]
[1. 0. 0. 0.]


[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[0.99 0.   0.01 0.  ]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]


[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[0.98 0.   0.02 0.  ]
[0.93 0.   0.07 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]

...

[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[0.96 0.   0.04 0.  ]
[0.87 0.   0.13 0.  ]
[0.38 0.   0.62 0.  ]
[0.97 0.   0.03 0.  ]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.96 0.   0.04 0.  ]
[0.92 0.   0.08 0.  ]


[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.98 0.   0.   0.02]
[1. 0. 0. 0.]
[0.98 0.   0.02 0.  ]
[0.94 0.01 0.05 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[0.97 0.   0.03 0.  ]
[0.95 0.   0.04 0.01]
[0.98 0.   0.02 0.  ]
[0.99 0.   0.01 0.  ]
[0.97 0.   0.02 0.01]
[0.98 0.   0.02 0.  ]
[0.97 0.   0.03 0.  ]
[0.99 0.   0.01 0.  ]


[0.87 0.   0.12 0.01]
[0.41 0.   0.59 0.  ]
[0.78 0.   0.2  0.02]
[0.35 0.   0.64 0.01]
[0.63 0.   0.37 0.  ]
[0.2  0.01 0.78 0.01]
[0.18 0.   0.82 0.  ]
[0.68 0.   0.32 0.  ]
[0.91 0.   0.09 0.  ]
[0.93 0.   0.07 0.  ]
[0.94 0.01 0.05 0.  ]
[0.95 0.02 0.03 0.  ]
[0.93 0.01 0.06 0.  ]
[0.93 0.   0.07 0.  ]
[0.85 0.   0.15 0.  ]
[0.26 0.   0.74 0.  ]
[0.86 0.   0.1  0.04]
[0.97 0.   0.02 0.01]
[0.97 0.   0.01 0.02]
[0.97 0.   0.02 0.01]
[0.98 0.01 0.01 0.  ]
[0.99 0.01 0.   0.  ]


[0.98 0.   0.02 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.   0.01]
[0.96 0.   0.01 0.03]
[1. 0. 0. 0.]
[1. 0. 0. 0.]
[0.97 0.01 0.02 0.  ]
[0.36 0.   0.03 0.61]
[0.14 0.   0.8  0.06]
[0.18 0.01 0.77 0.04]
[0.79 0.02 0.16 0.03]
[0.95 0.01 0.04 0.  ]
[1. 0. 0. 0.]
[0.99 0.   0.01 0.  ]


[0.95 0.04 0.01 0.  ]
[0.88 0.09 0.02 0.01]
[0.24 0.64 0.08 0.04]
